In [86]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
np.set_printoptions(precision=4, suppress=True)

F = 4500 # N
v = 3.05 # m/s
D = 500 # mm
L = 4 # year
t1 = 12 # s
t2 = 60 # s
T1 = 1 # N.mm
T2 = 0.7 # N.mm

## Motor Design $(power, rpm)$

In [51]:
# calculate efficiency, e = eta
e_coupling = 1
e_bearing = 0.99
e_helical_gear = 0.96
e_ch = 0.95
e_sys = e_coupling * e_bearing**3 * e_helical_gear * e_ch
print(e_sys)

P_max = F*v/1000 # calculate max power of the belt conveyor, kW
P_work = P_max*np.sqrt((T1**2*t1 + T2**2*t2)/(t1+t2)) # calculate power given the work load, kW
P_motor = P_work/e_sys # lower limit of the motor's power, kW
print(P_max, P_work, P_motor)

n_belt = v*6e4/np.pi/D # calculate rotational speed of the belt conveyor, rpm
u_ch = 5
u_hg = 5
u_sys = u_ch * u_hg # calculate transmission ratio of the system
n_motor = u_sys * n_belt # calculate theoretical rotational speed of the motor, rpm
print(n_belt, u_sys, n_motor)
P_motor = 18.5 # power based on choice of motor, kW
n_motor = 2930 # rotational speed based on choice of motor, rpm
u_sys = n_motor / n_belt # new transmission ratio
u_ch = u_sys / u_hg # new transmission ratio of ch drive, given u_hg=const
print(u_sys, u_ch)

P_ch = P_max
P_sh2 = P_ch / (e_ch*e_bearing) # power of shaft 2, kW
P_sh1 = P_sh2 / (e_helical_gear*e_bearing) # power of shaft 1, kW
P_motor = P_sh1 / (e_coupling) # power of motor, kW
n_sh1 = n_motor # rotational speed of shaft 1, rpm
n_sh2 = n_sh1 / (u_hg) # rotational speed of shaft 2, rpm
T_motor = 9.55e6 * P_motor / n_motor # torque of motor, N.mm
T_sh1 = 9.55e6 * P_sh1 / n_sh1 # torque of shaft 1, N.mm
T_sh2 = 9.55e6 * P_sh2 / n_sh2 # torque of shaft 2, N.mm
print(np.round([[P_sh2, P_sh1, P_motor],
                [n_motor, n_sh1, n_sh2],
                [T_motor, T_sh1, T_sh2]],2))

0.8849126879999999
13.725 10.407496546960752 11.761043420546766
116.50141834326739 25 2912.535458581685
25.14990840168878 5.029981680337756
[[    14.59     15.35     15.35]
 [  2930.     2930.      586.  ]
 [ 50047.56  50047.56 237825.99]]


## Chain Drive Design

In [52]:
# find z1, z2
z_1 = np.ceil(29 - 2*u_ch)
z_2 = np.ceil(u_ch*z_1) + 1 # eq (5.1), p.80, plus 1 to obtain odd number of teeth

# find k, k_z, k_n
n01 = 600 # ~ n_sh2
k_z, k_n = 25/z_1, n01/n_sh2 # eq (5.3), p.81
k_0, k_a, k_dc, k_bt, k_d, k_c = 1, 1, 1, 1, 1.25, 1.3
k = k_0 * k_a * k_dc * k_bt * k_d * k_c # table 5.6, p.82
P_allowance = P_ch * k * k_z * k_n # equals 30.047 <= 42
P_allowance, p, dc, B = 42, 31.75, 9.55, 27.46 # table 5.5, p.81
d_1 = p/np.sin(np.pi/z_1) # examine table 5.8, p<p_max~31.75
d_2 = p/np.sin(np.pi/z_2) # examine table 5.8, p<p_max~31.75
a_min, a_max, a = 30*p, 50*p, 1000 # 952.5<a<1587.5, choose a = 1000
x = 2*a/p + (z_1+z_2)/2 + (z_2-z_1)**2*p/(4*np.pi**2*a) # eq (5.12), p.85
x_c = np.ceil(x)
a = 0.25*p*(x_c-0.5*(z_2+z_1)+np.sqrt((x_c-0.5*(z_2+z_1))**2-2*((z_2-z_1)/np.pi)**2)) - 0.003*a # eq (5.13), p.85
i = z_1*n_sh2/(15*x) # eq (5.14), p.85; i<[i]=25 at table 5.9, p.85
B_min, d_O, d_l, h_max, b_max, Q, q = 15.88, 7.95, 15.88, 24.2, 38, 56.7, 2.6 # table 5.2, p.78
k_f, k_d = 4, 1.2
v_1 = n_sh2*p*z_1/6e4
F_t, F_v, F_O = 1e3*P_ch/v_1, q*v_1**2, 9.81*k_f*q*a/1e3
s = Q*1e3/(k_d*F_t + F_O + F_v)
s >= 10.3 # table 5.10, p.86
n1 = np.array([1250,1000,900,800,630,500,400,300])
pm = np.array([12.7,15.875,19.5,25.4,31.75,38.1,44.5,50.8])
f = interp1d(n1,pm,kind='cubic')
k_x = 1.15
F_r = k_x*F_t # eq (5.20), p.88
zv = 0.85*v_1**.1
kxh = 
print(np.round([i,v_1,F_t,F_v,F_O,s,F_r, F_O+F_v, F_O+F_v+F_t,zv],2))

[   5.9     5.89 2329.53   90.25  101.92   18.98 2678.96  192.17 2521.7 ]


In [190]:
H1, H2, NFO1, NFO2, Lh, alpha  = 250, 240, 4e6,4e6, 8*2*300*4, 20
NHO1, NHO2 = 30*H1**2.4, 30*H2**2.4
NHE1, NHE2 = 60*n_sh1*Lh*(T1**3*t1 + T2**3*t2)/(t1+t2), 60*n_sh2*Lh*(T1**3*t1 + T2**3*t2)/(t1+t2)
NFE1, NFE2 = 60*n_sh1*Lh*(T1**6*t1 + T2**6*t2)/(t1+t2), 60*n_sh2*Lh*(T1**6*t1 + T2**6*t2)/(t1+t2)
KHL1, KHL2 = (NHO1/NHE1)**(1/6), (NHO2/NHE2)**(1/6)
KFL1, KFL2 = (NFO1/NFE1)**(1/6), (NFO2/NFE2)**(1/6)
KHL1, KHL2, KFL1, KFL2 = 1,1,1,1
sigH1, sigH2 = 570/1.1*KHL1, 550/1.1*KHL2
sigF1, sigF2 = 450*KFL1/1.75, 432*KFL2/1.75
sigH = np.average([sigH1,sigH2])
print(np.round([NHE1, NHE2, NFE1, NFE2],2))
print(np.round([KHL1, KHL2, KFL1, KFL2, sigH1, sigH2, sigF1, sigF2, sigH],2))
psiba = 0.5
psibd = 0.53*psiba*(u_hg+1)
aw = 43*(u_hg+1)*(T_sh1*1.108/sigH**2/u_hg/psiba)**(1/3)
aw = 125
m, beta = 1.5, 14
z1 = 2*aw*np.cos(beta*np.pi/180) / (m*(u_hg+1))
print(z1)
z1 = 27
z2 = u_hg*z1
bw = psiba*aw
# fig, [ax1,ax2] = plt.subplots(nrows=2)
# ax1.plot(m*(z1+z2)/2/aw)
# ax2.plot(beta)
# print(np.arccos(min(m*(z1+z2)/2/aw))*180/np.pi)
# print(z1)
# print(bw,m*(z1+z2)/aw)
beta = np.arccos(m*(z1+z2)/2/aw)
d1 = dw1 = m*z1/np.cos(beta)
d2 = dw2 = m*z2/np.cos(beta)
da1 = d1+2*(1+0.3)*m
da2 = d2+2*(1-0.3)*m
df1 = d1-(2.5-2*0.3)*m
df2 = d2-(2.5-2*-0.3)*m
db1 = d1*np.cos(alpha*np.pi/180)
db2 = d2*np.cos(alpha*np.pi/180)
alpha_t = alpha_tw = np.arctan(np.tan(alpha*np.pi/180)/np.cos(beta))
v = np.pi*d1*n_sh1/6e4
zv = 0.85*v**.1
ys = 1.08-0.0695*np.log(m)
sigHH, sigFF1, sigFF2 = sigH*zv, sigF1*1.1*ys, sigF2*1.1*ys
print(np.round([d1,d2,da1,da2,df1,df2,db1,db2,
                alpha_t*180/np.pi,beta*180/np.pi,v,zv,ys,
               sigHH, sigFF1, sigFF2],2))

[1.5274e+09 3.0547e+08 8.9348e+08 1.7870e+08]
[  1.     1.     1.     1.   518.18 500.   257.14 246.86 509.09]
26.952659063222125
[ 41.67 208.33  45.57 210.43  38.82 203.68  39.15 195.77  20.53  13.59
   6.39   1.02   1.05 520.93 297.51 285.61]


In [169]:
beta_b = np.arctan(np.cos(alpha_t)*np.tan(beta))
zm, zh = 274, np.sqrt(2*np.cos(beta_b)/np.sin(2*alpha_tw))
epbeta = bw*np.sin(beta)/m/np.pi
epalpha = (np.sqrt(da1**2-db1**2)+np.sqrt(da2**2-db2**2)-2*aw*np.sin(alpha_tw))/(2*np.pi*m*np.cos(alpha_t)/np.cos(beta))
print(np.round([beta_b*180/np.pi,zh,epalpha,epbeta],2))
ze = epalpha**-0.5
kh, kf, mn = 1.07*1.1*1.108, 1.18*1.29*1.2558, m*np.cos(beta)
sigHc = zm*zh*ze*np.sqrt(2*T_sh1*kh*(u_hg+1)/bw/u_hg/dw1**2)
ye, yb = epalpha**-1, 1-beta*180/np.pi/140
zv1, zv2, yf1, yf2 = z1/np.cos(beta)**3, z2/np.cos(beta)**3, 4.0588, 3.5981
sigF1 = 2*T_sh1*kf*ye*yb*yf1/bw/dw1/mn
sigF2 = sigF1*yf2/yf1
Ft = 2*T_sh1/dw1
Fr = Ft*np.tan(alpha_tw)
Fa = Ft*np.tan(beta)
print(Ft,Fr,Fa)

[12.76  1.72  1.41  3.12]
2402.28270259134 899.5467057981642 580.7514733297112


## Shaft design

In [177]:
F_r = 2678.96
Ft1 = Ft2 = 2*T_sh1/dw1
Fr1 = Fr2 = Ft1*np.tan(alpha_t)/np.cos(beta)
Fa1 = Fa2 = Ft1*np.tan(beta)
r12, r21 = -dw1/2, dw2/2
hr12, cb12, cq1, hr21, cb21, cq2 = 1,1,1,-1,-1,-1
Fx12 = r12/abs(r12)*cq1*cb12*Ft1
Fx21 = r21/abs(r21)*cq2*cb21*Ft2
Fy12 = -r12/abs(r12)*Ft1*np.tan(alpha_t)/np.cos(beta)
Fy21 = -r21/abs(r21)*Ft2*np.tan(alpha_t)/np.cos(beta)
Fz12 = cq1*cb12*hr12*Ft1*np.tan(beta)
Fz21 = cq2*cb21*hr21*Ft2*np.tan(beta)
Fy22 = F_r*np.cos(210*np.pi/180)
Fx22 = F_r*np.sin(210*np.pi/180)
d1 = (T_sh1/.2/15)**(1/3)
d2 = (T_sh2/.2/30)**(1/3)
print(d1,d2)
d1, d2, bO1, bO2 = 30, 35, 19, 21
lm13,lm12,lm22,lm23,k1,k2,k3,hn= 1.5*d1,1.5*d1,1.5*d2,1.5*d2,10,8,15,18
print(np.round([lm13,lm12,lm22,lm23],2))
l22 = -(.5*(lm22+bO2)+k3+hn)
l23 = 0.5*(lm23+bO2)+k1+k2
l21 = 2*l23
l12 = -(.5*(lm12+bO1)+k3+hn)
l13 = 0.5*(lm13+bO1)+k1+k2
l11 = 2*l13
C2A2, A2D2, D2B2, A2B2 = -l22, l23, l23, l21
B1C1, A1D1, D1B1, A1B1 = -l12, l13, l13, l11
print(np.round([r12,r21,C2A2, A2D2, D2B2, A2B2,B1C1, A1D1, D1B1, A1B1],2))
print(np.round([Ft1,Fr1,Fa1,Fx12,Fy12,Fz12,Fx21,Fy21,Fz21,Fx22,Fy22],2))

25.55174359011898 34.09594141425404
[45.  45.  52.5 52.5]
[-20.83 104.17  69.75  54.75  54.75 109.5   65.    50.    50.   100.  ]
[ 2402.28   925.46   580.75 -2402.28   925.46   580.75  2402.28  -925.46
  -580.75 -1339.48 -2320.05]


In [183]:
from sympy import *
rA2y, rA2x, rB2y, rB2x, rA1y, rA1x, rB1y, rB1x = symbols('r_A2y, r_A2x, r_B2y, r_B2x, r_A1y, r_A1x, r_B1y, r_B1x')
eq1 = rA2x + rB2x + Fx21 + Fx22
eq2 = C2A2*rA2x + (C2A2+A2D2)*Fx21 + (C2A2+A2B2)*rB2x
rA2x,rB2x = np.array(list(linsolve([eq1,eq2],[rA2x,rB2x]))[0],dtype='float')
eq1 = rA2y + rB2y + Fy21 + Fy22
eq2 = C2A2*rA2y + (C2A2+A2D2)*Fy21 + (C2A2+A2B2)*rB2y + dw2*Fz21/2
rA2y,rB2y = np.array(list(linsolve([eq1,eq2],[rA2y,rB2y]))[0],dtype='float')
eq1 = rA1x + rB1x + Fx12
eq2 = A1D1*Fx12 + A1B1*rB1x
rA1x,rB1x = np.array(list(linsolve([eq1,eq2],[rA1x,rB1x]))[0],dtype='float')
eq1 = rA1y + rB1y + Fy12
eq2 = A1D1*Fy12 + A1B1*rB1y + dw1*Fz12/2
rA1y,rB1y = np.array(list(linsolve([eq1,eq2],[rA1y,rB1y]))[0],dtype='float')
# print(np.round([rA1x, rA1y, rB1x, rB1y, rA2x, rA2y, rB2x, rB2y],2))

MxA1, MxD1, MxB1, MxC1 = 0, A1D1*rA1x, 0, 0
MyA1, MyD1m, MyD1p, MyB1, MyC1 = 0, A1D1*rA1y, A1D1*rA1y - dw1*Fz12/2, A1B1*rA1y + D1B1*Fy12 - dw1*Fz12/2, 0
MxC2, MxA2, MxD2, MxB2 = 0, C2A2*Fx22, (C2A2+A2D2)*Fx22 + A2D2*rA2x, 0
MyC2, MyA2, MyD2m, MyD2p, MyB2 = 0, C2A2*Fy22, (C2A2+A2D2)*Fy22 + A2D2*rA2y, D2B2*rB2y,0

# print(np.round([MxA1, MxD1, MxB1, MxC1],2))
# print(np.round([MyA1, MyD1m, MyD1p, MyB1, MyC1],2))
# print(np.round([MxC2, MxA2, MxD2, MxB2],2))
# print(np.round([MyC2, MyA2, MyD2m, MyD2p, MyB2],2))
eqM = lambda mx, my, t: np.sqrt(mx**2 + my**2 + 0.75*t**2)
eq = lambda mx, my, t: np.sqrt(mx**2 + my**2)
MeA1 = eqM(MxA1,MyA1,T_sh1)
MeD1m = eqM(MxD1,MyD1m,T_sh1)
MeD1p = eqM(MxD1,MyD1p,T_sh1)
MeB1 = eqM(MxB1,MyB1,T_sh1)
MeC1 = eqM(MxC1,MyC1,T_sh1)
MeC2 = eqM(MxC2,MyC2,T_sh2)
MeA2 = eqM(MxA2,MyA2,T_sh2)
MeD2m = eqM(MxD2,MyD2m,T_sh2)
MeD2p = eqM(MxD2,MyD2p,T_sh2)
MeB2 = eqM(MxB2,MyB2,T_sh2)
MA1 = eq(MxA1,MyA1,T_sh1)
MD1m = eq(MxD1,MyD1m,T_sh1)
MD1p = eq(MxD1,MyD1p,T_sh1)
MB1 = eq(MxB1,MyB1,T_sh1)
MC1 = eq(MxC1,MyC1,T_sh1)
MC2 = eq(MxC2,MyC2,T_sh2)
MA2 = eq(MxA2,MyA2,T_sh2)
MD2m = eq(MxD2,MyD2m,T_sh2)
MD2p = eq(MxD2,MyD2p,T_sh2)
MB2 = eq(MxB2,MyB2,T_sh2)

Mess = np.array([MeA1,MeD1m,MeD1p,MeB1,MeC1,MeC2,MeA2,MeD2m,MeD2p,MeB2])
Mes1 = np.array([MeA1,MeD1p,MeB1,MeC1])
Mes2 = np.array([MeC2,MeA2,MeD2m,MeB2])
# print(np.round([rA1x,rA1y,rB1x,rB1y,rA2x,rA2y,rB2x,rB2y],2))
# print(np.round(Mess,2))
# print(np.round((np.array([Mes1/67,Mes2/64])/0.1)**(1/3),2))
print(np.round([MA1,MD1p,MB1,MC1,MC2,MA2,MD2m,MB2],2).reshape(8,1))
# Me2 = np.array([MeC2,MeA2,MeD2,MeB2])
# Me1 = np.array([MeA1,MeD1,MeB1,MeC1])
# print(np.round([(Mes1/0.1/67)**(1/3),(Mes2/0.1/64)**(1/3)],2))

[[     0.  ]
 [ 66773.3 ]
 [     0.  ]
 [     0.  ]
 [     0.  ]
 [186857.46]
 [141481.15]
 [     0.  ]]


## bearing design

In [187]:
X1,Y1,X2,Y2,lh,m1,m2 = .6,1.07,1.66,1,30000,3,3
Q1 = X1*abs(rA1y)+Y1*abs(Fz12)
Q2 = X2*abs(rB2y)+Y1*abs(Fz21)
L1 = lh*60*n_sh1*1e-6
L2 = lh*60*n_sh2*1e-6
Cd1 = Q1*L1**(1/m1)
Cd2 = Q2*L2**(1/m2)
print(Q1,Q2,L1,L2,Cd1,Cd2)

826.4480144565043 1389.3920922494046 5274.0 1054.8 14385.630985717533 14143.216727519313
